## 머신러닝 프로세스 기반 
### 수집된(전처리가된 데이터)-> 모델 -> 모델 학습 -> 예측(평가)
### 데이터 : UCI 레포지토리에서 제공이되는 , 유방암 진단 샘플 데이터 를 이용.
### 로지스틱 회귀, 시그모이드 함수,  
### 성능 평가 지표 -> 오차 행렬, 정밀도, 재현율 , F1 스코어 , ROC 기반 AUC 스코어 
### 예) 암이 아닌 환자, 암 환자 (예측)
### 행(실제)
### 0     TN            FP
### 1     FN            TP




## 용어정리 
### 로지스틱 회귀 : 분류에 사용하는 기법, 선형함수(직선)를 사용해서, 추세를 예측하는 선형회귀
### 선형 회귀랑은 다르게 , 참, 거짓을 분류를 하는 목적. 비선형함수(시그모이드 함수 )

### 시그모이드 함수 기본형,-> s자형이고, x 클수록 1에 근사하고, , x 작을수록 0에 근사합니다. 
### f(x) = 1 / 1+ e^-x

### 오차 행렬
### -> 사이킷런에 함수로 제공 : confusion_matrix

###                 Negative    Positive
### 행(실제 클래스),(예측 부정)  (예측 긍정)  열( 예측 클래스 )
### 
### (실제 부정)0     TN            FP
### (실제 긍정)1     FN            TP

### 정확도 = 바르게 예측하는 모든 수 (TN + TP) / 전체 데이터 수 (TN + FP + FN + TP)
### 정밀도 = 참인 것의 수 (TP) / 예측이 긍정인것(FP + TP)
### -> Positive 예측 성능을 더 정밀하게 평가하는 지표, 
### -> 사이킷런에 함수로 제공 : precision_score


### ex) 암 환자 진단할 때, 정확도 중요한가요?, 정밀도 중요한가요? , 정밀도가 더 중요함. 
### 암이냐?, 아니냐?  -> 정밀도, 

### 재현율 = 참인 것의 수 (TP) / 실제값 이 긍정인것 중에서 (FN + TP)
### -> 실제 Positive 인 데이터를 정확히 예측하는 평가 지표. 민감도라고하고, TPR(True Positive Rate)
### -> 사이킷런에 함수로 제공 : recall_score

### F1 스코어 : 정밀도와 재현율을 결합한 평가지표, 상충관계 문제점을 고려해서 , 정확한 평가에 사용이됨. 
### F1 스코어 = 2 x (정밀도 x 재현율)/(정밀도 + 재현율)

### ROC 기반 AUC 스코어 
### ROC 기반 AUC 스코어 = TPR / FPR
### FPR(False Positive Rate)  = 거짓인 것의 수 (FP) / 실제값 이 거짓인것 중에서 (TN + FP)
### ROC 기반 AUC 스코어 -> 1에 가까울수록 좋은 성능. 
### -> 사이킷런에 함수로 제공 : roc_auc_score


### 유방암 데이터 샘플을 가지고, 실제 암환자, 모델을 학습 -> 학습된 모델을 평가 
### 평가의 예측값이 -> 실제값과 어느정도 일치 하는지 여부, 우리가 확인. 

### 보스턴의 집값도, 사이킷런에서 내장된 데이터를 사용했음. 

In [1]:
import numpy as np 
import pandas as pd 

from sklearn.datasets import load_breast_cancer

In [3]:
b_cancer = load_breast_cancer()
print(b_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [5]:
b_cancer_df = pd.DataFrame(b_cancer.data, columns=b_cancer.feature_names)
# 새 컬럼 추가, 종속 변수인, 암의 진단 여부 컬럼 추가. 
b_cancer_df["diagnosis"] = b_cancer.target
b_cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0
